**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten


Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The Agent act as a greedy agent, playing the best play with probability $1-\epsilon$. Otherwise, it will 'explore' the environment by acting as random.

$\epsilon$ is a kind of 'exploration' parameter. If $\epsilon = 0$, you will have a greedy algorithm that will not learn the environment, while with $\epsilon=1$, the algorithm will learn the environment but with no taking in account what he learned.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [28]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```board``` contain the reward for each position. ```position``` says if a given position has been explored or not by the rat. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [13]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    for e in range(epochs):
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward >= 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        # Save as a mp4
        env.draw(prefix+str(e))
        
        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [12]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 5.5/9.0. Average score (-3.5)
Win/lose count 11.0/24.0. Average score (-8.25)
Win/lose count 8.5/12.0. Average score (-6.666666666666667)
Final score: -6.666666666666667


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We can do the following calculus, from the definition of the $Q$-function:

\begin{align*}Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
&= E_{p^{\pi}}[r(s,a) + \sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] (1)\\
&= r(s,a) + E_{p^{\pi}}[\sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] (2)\\
&= r(s,a) + \sum_{(s',a')}p(s',a'|s,a)*E_{p^{\pi}}\left[\sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'\right] (3)\\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)] + \gamma \sum_{(s',a')}p(s',a'|s,a)*E_{p^{\pi}} \left[\sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'\right] (4)\\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)] + \gamma E_{(s',a')\sim p(.|s,a)} \left[E_{p^{\pi}}\left[\sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'\right]\right] (5)\\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] (6)
\end{align*}



Explanations:
* (1): just decomposing the sum inside expectation
* (2): r(s,a) is deterministic
* (3): Rewriting the expectation, in order to be conditional to the following state, using the definition of $p_\pi$
* (4): r(s,a) is deterministic and can be put inside conditional expectation
* (5): Recongnizing the conditional expectation: $E_{(s',a')\sim p(.|s,a)}$
* (6): Recognizing the Q-function

For the optimal policy, we have:
\begin{align*}
Q^*(s,a) &= \max_{\pi}Q^\pi(s,a) \\
&= E_{(s',a')\sim \pi(.|a)*p(.|s,a)}[r(s,a)+\gamma Q^{*}(s',a')] \\
\end{align*}
We know that the optimal policy is such that $p(.|s,a) = \pi^*(.|s,a)$. We then conclude.

To conclude about the loss, it just comes from the fact that $Q(s,a,\theta)=E_{s' \sim \pi(s,a)}[Q(s,a,\theta)]$, because it is deterministic, so we can choose that loss which is almost $||Q-Q^*||^2$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [7]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if self.max_memory < len(self.memory):
            self.memory.pop(0)

    def random_access(self):
        i = np.random.randint(0,len(self.memory))
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [8]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [9]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.reshape(s,(1,5,5,)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        for i in range(self.batch_size):
            [s_, n_s_, a_, r_, game_over_] = self.memory.random_access()
            target_q[i,:] = self.model.predict(s_)
            input_states[i,:,:,:] = s_
            if game_over_:
                target_q[i,a_] = r_
            else:
                target_q[i,a_] = r_ + self.discount*np.amax(self.model.predict(n_s_))
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(24))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dense(24))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(Adam(lr=lr), "mse")
        self.model = model
        

In [55]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)

epochs_train=201
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train200.mp4'))

Epoch 000/201 | Loss 0.0014 | Win/lose count 4.5/0 (4.5)
Epoch 001/201 | Loss 0.0047 | Win/lose count 1.0/6.0 (-5.0)
Epoch 002/201 | Loss 0.0082 | Win/lose count 5.0/1.0 (4.0)
Epoch 003/201 | Loss 0.0073 | Win/lose count 3.0/5.0 (-2.0)
Epoch 004/201 | Loss 0.0116 | Win/lose count 4.5/5.0 (-0.5)
Epoch 005/201 | Loss 0.0144 | Win/lose count 5.0/5.0 (0.0)
Epoch 006/201 | Loss 0.0097 | Win/lose count 5.5/9.0 (-3.5)
Epoch 007/201 | Loss 0.0070 | Win/lose count 7.0/8.0 (-1.0)
Epoch 008/201 | Loss 0.0041 | Win/lose count 4.5/2.0 (2.5)
Epoch 009/201 | Loss 0.0051 | Win/lose count 5.0/2.0 (3.0)
Epoch 010/201 | Loss 0.0061 | Win/lose count 7.0/1.0 (6.0)
Epoch 011/201 | Loss 0.0359 | Win/lose count 11.5/2.0 (9.5)
Epoch 012/201 | Loss 0.0019 | Win/lose count 8.5/6.0 (2.5)
Epoch 013/201 | Loss 0.0075 | Win/lose count 7.0/5.0 (2.0)
Epoch 014/201 | Loss 0.0056 | Win/lose count 6.0/3.0 (3.0)
Epoch 015/201 | Loss 0.0061 | Win/lose count 10.5/10.0 (0.5)
Epoch 016/201 | Loss 0.0606 | Win/lose count 8.0/4

Epoch 138/201 | Loss 0.0022 | Win/lose count 16.5/5.0 (11.5)
Epoch 139/201 | Loss 0.0009 | Win/lose count 5.5/4.0 (1.5)
Epoch 140/201 | Loss 0.0010 | Win/lose count 12.0/5.0 (7.0)
Epoch 141/201 | Loss 0.0018 | Win/lose count 3.0/3.0 (0.0)
Epoch 142/201 | Loss 0.0022 | Win/lose count 12.5/2.0 (10.5)
Epoch 143/201 | Loss 0.0015 | Win/lose count 8.0/4.0 (4.0)
Epoch 144/201 | Loss 0.0011 | Win/lose count 13.0/5.0 (8.0)
Epoch 145/201 | Loss 0.0028 | Win/lose count 4.0/2.0 (2.0)
Epoch 146/201 | Loss 0.0013 | Win/lose count 12.0/3.0 (9.0)
Epoch 147/201 | Loss 0.0019 | Win/lose count 13.5/2.0 (11.5)
Epoch 148/201 | Loss 0.0025 | Win/lose count 10.0/3.0 (7.0)
Epoch 149/201 | Loss 0.0011 | Win/lose count 16.0/5.0 (11.0)
Epoch 150/201 | Loss 0.0021 | Win/lose count 15.5/5.0 (10.5)
Epoch 151/201 | Loss 0.0009 | Win/lose count 2.0/1.0 (1.0)
Epoch 152/201 | Loss 0.0550 | Win/lose count 6.5/1.0 (5.5)
Epoch 153/201 | Loss 0.0013 | Win/lose count 16.5/3.0 (13.5)
Epoch 154/201 | Loss 0.0049 | Win/lose c

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [42]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        
        model.add(Conv2D(8,(3,3),padding='same'))
        
        model.add(Activation('relu'))
        
        model.add(Conv2D(16,(3,3),padding='same'))
        
        model.add(Activation('relu'))
        model.add(MaxPooling2D())
        
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        
        
        model.compile(Adam(lr=lr), "mse")
        self.model = model

In [43]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
epochs_train=201 
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train200.mp4'))

Epoch 000/201 | Loss 0.0045 | Win/lose count 7.0/8.0 (-1.0)
Epoch 001/201 | Loss 0.0097 | Win/lose count 6.5/8.0 (-1.5)
Epoch 002/201 | Loss 0.0094 | Win/lose count 4.0/7.0 (-3.0)
Epoch 003/201 | Loss 0.0028 | Win/lose count 5.5/5.0 (0.5)
Epoch 004/201 | Loss 0.0031 | Win/lose count 2.0/2.0 (0.0)
Epoch 005/201 | Loss 0.0072 | Win/lose count 5.0/4.0 (1.0)
Epoch 006/201 | Loss 0.0046 | Win/lose count 9.5/3.0 (6.5)
Epoch 007/201 | Loss 0.0070 | Win/lose count 7.0/4.0 (3.0)
Epoch 008/201 | Loss 0.0484 | Win/lose count 5.0/1.0 (4.0)
Epoch 009/201 | Loss 0.0019 | Win/lose count 12.5/1.0 (11.5)
Epoch 010/201 | Loss 0.0092 | Win/lose count 2.5/4.0 (-1.5)
Epoch 011/201 | Loss 0.0019 | Win/lose count 6.0/8.0 (-2.0)
Epoch 012/201 | Loss 0.0013 | Win/lose count 3.5/2.0 (1.5)
Epoch 013/201 | Loss 0.0017 | Win/lose count 6.5/5.0 (1.5)
Epoch 014/201 | Loss 0.0550 | Win/lose count 3.5/1.0 (2.5)
Epoch 015/201 | Loss 0.0075 | Win/lose count 9.5/4.0 (5.5)
Epoch 016/201 | Loss 0.0041 | Win/lose count 11.5

Epoch 138/201 | Loss 0.0027 | Win/lose count 5.5/1.0 (4.5)
Epoch 139/201 | Loss 0.0083 | Win/lose count 16.5/2.0 (14.5)
Epoch 140/201 | Loss 0.0029 | Win/lose count 11.5/1.0 (10.5)
Epoch 141/201 | Loss 0.0496 | Win/lose count 19.0/3.0 (16.0)
Epoch 142/201 | Loss 0.0031 | Win/lose count 10.0/4.0 (6.0)
Epoch 143/201 | Loss 0.0021 | Win/lose count 14.0/8.0 (6.0)
Epoch 144/201 | Loss 0.0539 | Win/lose count 15.5/0 (15.5)
Epoch 145/201 | Loss 0.0018 | Win/lose count 7.5/2.0 (5.5)
Epoch 146/201 | Loss 0.0019 | Win/lose count 5.5/5.0 (0.5)
Epoch 147/201 | Loss 0.0037 | Win/lose count 22.5/2.0 (20.5)
Epoch 148/201 | Loss 0.0007 | Win/lose count 12.0/2.0 (10.0)
Epoch 149/201 | Loss 0.0043 | Win/lose count 13.0/4.0 (9.0)
Epoch 150/201 | Loss 0.0040 | Win/lose count 7.5/4.0 (3.5)
Epoch 151/201 | Loss 0.0030 | Win/lose count 15.5/5.0 (10.5)
Epoch 152/201 | Loss 0.0010 | Win/lose count 5.5/1.0 (4.5)
Epoch 153/201 | Loss 0.0035 | Win/lose count 8.0/1.0 (7.0)
Epoch 154/201 | Loss 0.0014 | Win/lose co

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [44]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
epochs_test=11
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.0/2.0. Average score (4.0)
Win/lose count 4.5/1.0. Average score (3.75)
Win/lose count 7.5/3.0. Average score (4.0)
Win/lose count 4.0/0. Average score (4.0)
Win/lose count 11.0/5.0. Average score (4.4)
Win/lose count 4.5/1.0. Average score (4.25)
Win/lose count 4.0/1.0. Average score (4.071428571428571)
Win/lose count 8.5/3.0. Average score (4.25)
Win/lose count 8.0/1.0. Average score (4.555555555555555)
Win/lose count 6.0/2.0. Average score (4.5)
Win/lose count 5.5/3.0. Average score (4.318181818181818)
Final score: 4.318181818181818
Test of the FC
Win/lose count 3.0/0. Average score (3.0)
Win/lose count 6.5/3.0. Average score (3.25)
Win/lose count 7.0/2.0. Average score (3.8333333333333335)
Win/lose count 15.0/3.0. Average score (5.875)
Win/lose count 13.0/4.0. Average score (6.5)
Win/lose count 13.0/6.0. Average score (6.583333333333333)
Win/lose count 17.5/2.0. Average score (7.857142857142857)
Win/lose count 5.0/5.0. Average score (6.875)
Win/lose

In [45]:
HTML(display_videos('cnn_test10.mp4'))

In [21]:
HTML(display_videos('fc_test10.mp4'))

There is two problems here.
* First, the Agent will not explore if he has eaten all the cheese around him. He will play too much secure, and will wait until the $\epsilon$ test gets a success to explore
* The $\epsilon$ exploration becomes a problem when the agent starts to be good, and makes the agent eat some poison, while the greedy strategy could have make him eat some cheese

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [52]:
def train_explore(agent,env,epoch,prefix=''):
       # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((grid_size,grid_size))


    def act(self, action,train = False):
        """This function returns the new state, reward and decides if the
        game ends."""
        state,reward,game_over = super(EnvironmentExploring,self).act(action)
        self.malus_position[self.x,self.y] += 10
        if train:
            reward = -self.malus_position[self.x, self.y]
        malus = self.malus_position[self.x-2:self.x+3,self.y-2:self.y+3]
        state = np.concatenate((state,
                        malus.reshape(5, 5,1)),axis=2)
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        state = super(EnvironmentExploring,self).reset()
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        malus = self.malus_position[self.x-2:self.x+3,self.y-2:self.y+3]
        state = np.concatenate((state,
                        malus.reshape(5, 5,1)),axis=2)
        return state

In [53]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 64,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore200.mp4'))

Epoch 000/201 | Loss 0.3645 | Win/lose count 0/7640.0 (-7640.0)
Epoch 001/201 | Loss 0.2060 | Win/lose count 0/6280.0 (-6280.0)
Epoch 002/201 | Loss 0.1442 | Win/lose count 0/8610.0 (-8610.0)
Epoch 003/201 | Loss 0.1058 | Win/lose count 0/7030.0 (-7030.0)
Epoch 004/201 | Loss 0.0830 | Win/lose count 0/8490.0 (-8490.0)
Epoch 005/201 | Loss 0.0444 | Win/lose count 0/6970.0 (-6970.0)
Epoch 006/201 | Loss 0.0520 | Win/lose count 0/6090.0 (-6090.0)
Epoch 007/201 | Loss 0.0644 | Win/lose count 0/21060.0 (-21060.0)
Epoch 008/201 | Loss 0.0285 | Win/lose count 0/22190.0 (-22190.0)
Epoch 009/201 | Loss 0.0186 | Win/lose count 0/4730.0 (-4730.0)
Epoch 010/201 | Loss 0.0203 | Win/lose count 0/10210.0 (-10210.0)
Epoch 011/201 | Loss 0.0112 | Win/lose count 0/7760.0 (-7760.0)
Epoch 012/201 | Loss 0.0118 | Win/lose count 0/9620.0 (-9620.0)
Epoch 013/201 | Loss 0.0111 | Win/lose count 0/5490.0 (-5490.0)
Epoch 014/201 | Loss 0.0075 | Win/lose count 0/8410.0 (-8410.0)
Epoch 015/201 | Loss 0.0075 | Win/

Epoch 128/201 | Loss 0.0000 | Win/lose count 0/7270.0 (-7270.0)
Epoch 129/201 | Loss 0.0000 | Win/lose count 0/5930.0 (-5930.0)
Epoch 130/201 | Loss 0.0000 | Win/lose count 0/5370.0 (-5370.0)
Epoch 131/201 | Loss 0.0000 | Win/lose count 0/4910.0 (-4910.0)
Epoch 132/201 | Loss 0.0000 | Win/lose count 0/5550.0 (-5550.0)
Epoch 133/201 | Loss 0.0000 | Win/lose count 0/7720.0 (-7720.0)
Epoch 134/201 | Loss 0.0000 | Win/lose count 0/4080.0 (-4080.0)
Epoch 135/201 | Loss 0.0000 | Win/lose count 0/4050.0 (-4050.0)
Epoch 136/201 | Loss 0.0000 | Win/lose count 0/7030.0 (-7030.0)
Epoch 137/201 | Loss 0.0000 | Win/lose count 0/4330.0 (-4330.0)
Epoch 138/201 | Loss 0.0000 | Win/lose count 0/6140.0 (-6140.0)
Epoch 139/201 | Loss 0.0000 | Win/lose count 0/6130.0 (-6130.0)
Epoch 140/201 | Loss 0.0000 | Win/lose count 0/6640.0 (-6640.0)
Epoch 141/201 | Loss 0.0000 | Win/lose count 0/6570.0 (-6570.0)
Epoch 142/201 | Loss 0.0000 | Win/lose count 0/12080.0 (-12080.0)
Epoch 143/201 | Loss 0.0000 | Win/lose

In [54]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 2.5/0. Average score (2.5)
Win/lose count 4.0/2.0. Average score (2.25)
Win/lose count 2.5/1.0. Average score (2.0)
Win/lose count 1.0/2.0. Average score (1.25)
Win/lose count 3.0/1.0. Average score (1.4)
Win/lose count 4.0/3.0. Average score (1.3333333333333333)
Win/lose count 1.5/1.0. Average score (1.2142857142857142)
Win/lose count 5.0/4.0. Average score (1.1875)
Win/lose count 1.5/2.0. Average score (1.0)
Win/lose count 2.5/1.0. Average score (1.05)
Win/lose count 2.5/3.0. Average score (0.9090909090909091)
Final score: 0.9090909090909091


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***